<a href="https://colab.research.google.com/github/erichsiao1106/cnn/blob/master/lymphoma_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
from glob import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import keras
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten
from keras.models import Model
from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
IMG_SIZE = 200
BATCH_SIZE = 32
class_map = {'CLL':0, 'MCL': 1, 'FL': 2}

In [ ]:
# Read single image
data_path = 'drive/My Drive/北分署/week8/lymphoma/*/*.tif' 
img_paths = glob(data_path)
img_paths_train, img_paths_test = train_test_split(img_paths, test_size=0.2)
len(img_paths), len(img_paths_train), len(img_paths_test)

In [ ]:
def data_generator(data_paths, batch_size, num_classes):
    '''data generator for fit_generator'''
    n = len(data_paths)
    i = 0
    data_paths = data_paths
    while True:
        image_data = []
        class_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(data_paths)
            path = data_paths[i]
            img = cv2.imread(path)
            img_resize = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img_resize = img_resize/255
            
            cls = path.split('/')[-2]

            image_data.append(img_resize)
            class_data.append(class_map[cls])
            i = (i+1) % n
        image_data = np.array(image_data)
        class_data = np.array(class_data)
        class_data = keras.utils.to_categorical(class_data, num_classes=num_classes)
        yield image_data, class_data


In [ ]:
generator_train = data_generator(data_paths=img_paths_train, batch_size=BATCH_SIZE, num_classes=3)
generator_test = data_generator(data_paths=img_paths_test, batch_size=BATCH_SIZE, num_classes=3)

In [ ]:
# 1
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = Conv2D(filters=16, kernel_size=3, activation='relu')(inputs)
x = Conv2D(filters=16, kernel_size=3, activation='relu')(x)
x = MaxPool2D(2)(x)
x = Conv2D(filters=16, kernel_size=3, activation='relu')(x)
x = Conv2D(filters=16, kernel_size=3, activation='relu')(x)
x = MaxPool2D(2)(x)
x = Flatten()(x)
x = Dense(64, activation=keras.activations.relu)(x)
prediction = Dense(3, activation='softmax')(x)
model = Model(inputs=inputs, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
logs = model.fit_generator(generator_train,
                           epochs=10,
                           steps_per_epoch=len(img_paths_train)//BATCH_SIZE,
                           validation_data=generator_test,
                           validation_steps=len(img_paths_test)//BATCH_SIZE,
                            )

In [ ]:
history = logs.history

In [ ]:
plt.plot(history['acc'])
plt.plot(history['val_acc'])
plt.legend(['acc', 'val_acc'])
plt.title('acc')

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.title('loss')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = np.argmax(y_test, axis=-1)
y_pred = np.argmax(model.predict(X_test), axis=-1)
print(y_true.shape, y_pred.shape)
target_names = [str(i) for i in range(3)]
print(classification_report(y_true, y_pred, target_names=target_names))
print(confusion_matrix(y_true, y_pred))
